In [1]:
import pandas as pd
import torch

diabetes_df = pd.read_csv("diabetes.csv")
diabetes_df.head()

,Pregnancies,Glucose,BloodPressure,SkinThickness,Insulin,BMI,DiabetesPedigreeFunction,Age,Outcome
0,6,148,72,35,0,33.6,0.627,50,1
1,1,85,66,29,0,26.6,0.351,31,0
2,8,183,64,0,0,23.3,0.672,32,1
3,1,89,66,23,94,28.1,0.167,21,0
4,0,137,40,35,168,43.1,2.288,33,1


In [2]:
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler

X = diabetes_df.drop('Outcome', axis=1).values
y = diabetes_df['Outcome'].values

# Split into training and test set
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.2, random_state=42, stratify=y)

# #Standardize
sc= StandardScaler()
X_train=sc.fit_transform(X_train)
X_test=sc.fit_transform(X_test)

In [3]:
import torch.nn as nn
import torch.nn.functional as F #this has activation functions

# Creating tensors
X_train = torch.FloatTensor(X_train)
X_test = torch.FloatTensor(X_test)

y_train = torch.LongTensor(y_train)
y_test = torch.LongTensor(y_test)

print(X_train)

tensor([[-0.8514, -0.9801, -0.4048,  ..., -0.6077,  0.3108, -0.7922],
        [ 0.3566,  0.1614,  0.4654,  ..., -0.3021, -0.1164,  0.5610],
        [-0.5494, -0.5045, -0.6223,  ...,  0.3726, -0.7649, -0.7076],
        ...,
        [-0.8514, -0.7582,  0.0303,  ...,  0.7800, -0.7861, -0.2847],
        [ 1.8665, -0.3142,  0.0303,  ..., -0.5695, -1.0194,  0.5610],
        [ 0.0546,  0.7322, -0.6223,  ..., -0.3149, -0.5770,  0.3073]])


In [4]:
class ANN_Model(nn.Module):
    def __init__(self, input_features=8, hidden1=20, hidden2=20, out_features =2):
        super().__init__()
        self.layer_1_connection = nn.Linear(input_features, hidden1)
        self.layer_2_connection = nn.Linear(hidden1, hidden2)
        self.out = nn.Linear(hidden2, out_features)
    
    def forward(self, x):
        #apply activation functions
        x = F.relu(self.layer_1_connection(x))
        x = F.relu(self.layer_2_connection(x))
        x = self.out(x)
        return x

In [5]:
torch.manual_seed(42)

#instantiate the model
model = ANN_Model()

In [6]:
# loss function
loss_function = nn.CrossEntropyLoss()

#optimizer
optimizer = torch.optim.AdamW(model.parameters(), lr = 0.01)

In [7]:
#run model through multiple epochs/iterations
final_loss = []
n_epochs = 500
for epoch in range(n_epochs):
    y_pred = model.forward(X_train)
    loss = loss_function(y_pred, y_train)
    final_loss.append(loss)
    
    if epoch % 10 == 1:
        print(f'Epoch number: {epoch} with loss: {loss.item()}')
    
    optimizer.zero_grad() #zero the gradient before running backwards propagation
    loss.backward() #for backward propagation 
    optimizer.step() #performs one optimization step each epoch
    

Epoch number: 1 with loss: 0.6474166512489319
Epoch number: 11 with loss: 0.5238223671913147
Epoch number: 21 with loss: 0.45228567719459534
Epoch number: 31 with loss: 0.41647645831108093
Epoch number: 41 with loss: 0.3912210166454315
Epoch number: 51 with loss: 0.3679707646369934
Epoch number: 61 with loss: 0.342700332403183
Epoch number: 71 with loss: 0.3138030469417572
Epoch number: 81 with loss: 0.28234538435935974
Epoch number: 91 with loss: 0.2524298131465912
Epoch number: 101 with loss: 0.22721917927265167
Epoch number: 111 with loss: 0.20655879378318787
Epoch number: 121 with loss: 0.1846778392791748
Epoch number: 131 with loss: 0.16674774885177612
Epoch number: 141 with loss: 0.14951631426811218
Epoch number: 151 with loss: 0.13775938749313354
Epoch number: 161 with loss: 0.12599745392799377
Epoch number: 171 with loss: 0.11742440611124039
Epoch number: 181 with loss: 0.10884484648704529
Epoch number: 191 with loss: 0.10019759833812714
Epoch number: 201 with loss: 0.094867780

In [8]:
#predictions
y_pred = []

with torch.no_grad():
    for i, data in enumerate(X_test):
        prediction = model(data)
        y_pred.append(prediction.argmax().item())



In [9]:
from sklearn.metrics import accuracy_score
a_score = accuracy_score(y_test, y_pred)
print(a_score)

0.7337662337662337


In [10]:
from sklearn.metrics import classification_report

print(classification_report(y_test, y_pred))

              precision    recall  f1-score   support

           0       0.83      0.74      0.78       100
           1       0.60      0.72      0.66        54

    accuracy                           0.73       154
   macro avg       0.72      0.73      0.72       154
weighted avg       0.75      0.73      0.74       154



Scores were substantially lower with SGD. Scores were somewhat higher with AdamW. 

With SGD, loss only decreased slightly, from 0.6612 at epoch 1 to 0.5982 at epoch 491. Accuracy, precision, recall, and f1 scores ranged from 0.32 to 0.65, so some were very low and none were especially high. 

Alternatively, with AdamW and to a slightly lesser extent with Adam, loss decreased drastically every 10 epochs. Using the Adam W optimizer, loss was reduced from 0.6474 at epoch 1 to 0.0085 at epoch 491. Accuracy, precision, recall, and F1 scores were all at or above 0.7. Using the Adam optimizer, loss was reduced even more (from 0.6474 at epoch 1 to 0.0046 at epoch 491). However, scores were slightly lower, though still at or above 0.67 across the board. 

In [11]:
def divisors(number):
    divisor_list = []
    for i in range(1, number + 1):
        if number % i == 0:
            divisor_list.append(i)
        else:
            pass
    if len(divisor_list) == 1:
        converted_list = [str(element) for element in divisor_list]
        print("There is", len(divisor_list), "divisor:", "".join(converted_list))
    elif len(divisor_list) == 2:
        converted_list = [str(element) for element in divisor_list]
        print("There are", len(divisor_list), "divisors:", " and ".join(converted_list))
    else:
        converted_list = [str(element) for element in divisor_list]
        print("There are", len(divisor_list), "divisors:", ", ".join(converted_list[:-1]) + ", and " + converted_list[-1])

In [12]:
divisors(1)

There is 1 divisor: 1


In [13]:
divisors(5)

There are 2 divisors: 1 and 5


In [14]:
divisors(40)

There are 8 divisors: 1, 2, 4, 5, 8, 10, 20, and 40
